In [1]:
#A) Clearly write outcome of data analysis
from zipfile import ZipFile

with ZipFile('blogs.zip', 'r') as zipdata:
    data_csv = zipdata.open('blogtext.csv')

In [2]:
import pandas as pd

df = pd.read_csv(data_csv)

In [3]:
del data_csv

In [4]:
df.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [5]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [6]:
df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [7]:
df.describe

<bound method NDFrame.describe of              id gender  age              topic      sign          date  \
0       2059027   male   15            Student       Leo   14,May,2004   
1       2059027   male   15            Student       Leo   13,May,2004   
2       2059027   male   15            Student       Leo   12,May,2004   
3       2059027   male   15            Student       Leo   12,May,2004   
4       3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   
...         ...    ...  ...                ...       ...           ...   
681279  1713845   male   23            Student    Taurus  01,July,2004   
681280  1713845   male   23            Student    Taurus  01,July,2004   
681281  1713845   male   23            Student    Taurus  01,July,2004   
681282  1713845   male   23            Student    Taurus  01,July,2004   
681283  1713845   male   23            Student    Taurus  01,July,2004   

                                                     text  
0                

In [8]:
#B) Clean the Structured Data
#i) Missing value analysis and imputation
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [9]:
#ii) Eliminate Non-English textual data

In [10]:
#cut the data for less execution time
df = df.head(3000)

In [11]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import reuters
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
!pip install nltk

In [13]:
df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
2995,589736,male,35,Technology,Aries,"05,August,2004",but that zoo exhibit thing was much...
2996,589736,male,35,Technology,Aries,"05,August,2004",my fave song for the day: Aimee Man...
2997,589736,male,35,Technology,Aries,"05,August,2004",urlLink America's Best Zoo Exhibit...
2998,589736,male,35,Technology,Aries,"05,August,2004",'The less one makes declaritive sta...


In [14]:
!pip install langdetect

In [15]:
from langdetect import detect

In [16]:
def detect_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [17]:
df = df[df['text'].apply(detect_english)]

In [18]:
df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
...,...,...,...,...,...,...,...
2994,589736,male,35,Technology,Aries,"05,August,2004","hey, how is everyone doing? i want..."
2995,589736,male,35,Technology,Aries,"05,August,2004",but that zoo exhibit thing was much...
2996,589736,male,35,Technology,Aries,"05,August,2004",my fave song for the day: Aimee Man...
2998,589736,male,35,Technology,Aries,"05,August,2004",'The less one makes declaritive sta...


In [19]:
#2) Preprocess unstructured data to make it consumable for model training.
# A) Eliminate All special Characters and Numbers
# Select only alphabets
import re
df.text = df.text.apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

F:\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
# B) Lowercase all textual data
df.text = df.text.apply(lambda x: x.lower())

In [21]:
# C)Remove all Stopwords
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [22]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
df.text = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

In [23]:
#D)Remove all extra white spaces
df.text = df.text.apply(lambda x: x.strip())

In [24]:
df.text[5]

'interesting conversation dad morning talking koreans put money invariably lot real estate cash cash would include short term investments one year well savings accounts reason real estate makes money lot money seen surveys seoul real estate rising per year long stretches even taking account crisis referred imf crisis although imf bailed korea compare korean corporate bonds fell modestly recovered local stock market represented kospi version dow jones index gone appreciably high points points see urllink link see real estate makes sense back conversation noted real big elite real estate investor billion usd see urllink converter properties dad seemed little flabbergasted heck need million dollars need much retire maybe lot risk take real estate south korean asset example north toots horn louder make move country usd worth cents also denominated imf crisis dropped vis vis usd also make bad investment fall victim scam latest urllink good morning city project toast saw lady tv lost everyth

In [25]:
#3) Build a base Classification model
#A)Create dependent and independent variables
df['labels'] = df.apply(lambda row: [row['gender'], str(row['age']), row['topic'], row['sign']], axis=1)

C:\Users\HP\AppData\Local\Temp/ipykernel_97244/3706867643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = df.apply(lambda row: [row['gender'], str(row['age']), row['topic'], row['sign']], axis=1)


In [26]:
df.head(5)

,id,gender,age,topic,sign,date,text,labels
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[male, 15, Student, Leo]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",interesting conversation dad morning talking k...,"[male, 33, InvestmentBanking, Aquarius]"


In [27]:
df = df[['text','topic']]

In [28]:
df

,text,topic
0,info found pages mb pdf files wait untill team...,Student
2,het kader van kernfusie op aarde maak je eigen...,Student
3,testing testing,Student
4,thanks yahoo toolbar capture urls popups means...,InvestmentBanking
5,interesting conversation dad morning talking k...,InvestmentBanking
...,...,...
2994,hey everyone want go game still looking job st...,Technology
2995,zoo exhibit thing mucho mucho funny,Technology
2996,fave song day aimee mann pavlov bell album los...,Technology
2998,less one makes declaritive statements less apt...,Technology


In [29]:
#B) Split data into train and test.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text.values, df.topic.values, test_size=0.20, random_state=42)

In [30]:
#C) Vectorize data using any one vectorizer.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [31]:
vectorizer.get_feature_names()[:12]

F:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aa',
 'aa anger',
 'aa compared',
 'aa nice',
 'aaa',
 'aaa take',
 'aaa travel',
 'aaaaaah',
 'aaaaack',
 'aaaah',
 'aaaah wisdom',
 'aaagh']

In [32]:
X_train_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [33]:
topic_counts = dict()

for topic in df.topic.values:
    for topic in topic:
        if topic in topic_counts:
            topic_counts[topic] += 1
        else:
            topic_counts[topic] = 1

In [34]:
topic_counts

{'S': 500,
 't': 1281,
 'u': 531,
 'd': 952,
 'e': 2534,
 'n': 3749,
 'I': 90,
 'v': 79,
 's': 203,
 'm': 100,
 'B': 95,
 'a': 307,
 'k': 522,
 'i': 1092,
 'g': 1801,
 'U': 436,
 'N': 45,
 'o': 3336,
 '-': 136,
 'P': 45,
 'r': 349,
 'f': 45,
 'E': 235,
 'c': 1753,
 'C': 14,
 'M': 16,
 'p': 75,
 'R': 75,
 'A': 4,
 'L': 2,
 'b': 2,
 'T': 1475,
 'h': 1475,
 'l': 1475,
 'y': 1475}

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=sorted(topic_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)

In [36]:
#D) Build a base model for Supervised Learning - Classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

In [37]:
clf.fit(X_train_bow, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [38]:
predicted_topic = clf.predict(X_test_bow)
predicted_scores = clf.decision_function(X_test_bow)

In [39]:
pred_inversed = mlb.inverse_transform(predicted_topic)
y_test_inversed = mlb.inverse_transform(y_test)

In [40]:
for i in range(5):
    print('Title:\t{}\nTrue topic:\t{}\nPredicted topic:\t{}\n\n'.format(
        X_test[i],
        ','.join(y_test_inversed[i]),
        ','.join(pred_inversed[i])
    ))

Title:	sharp ring course liked ring
True topic:	T,c,e,g,h,l,n,o,y
Predicted topic:	T,c,e,g,h,l,n,o,y


Title:	crisp cold morning certainly wake much tongues flame
True topic:	T,c,e,g,h,l,n,o,y
Predicted topic:	T,c,e,g,h,l,n,o,y


Title:	circle irony welcomes two latest members shannon erwin johnathan better late never shipley please see prior notes proper blogging contained prior posts archive section good reference questions episode news apparently several years e e years e e luke leia appear infants e towards end rant getting tired cgi used create human human like creatures landscapes ok cgi creatures still fancy cartoon characters indistinguishable actual creatures take seriously e serve clever distracting talking head might otherwise serious scene making comedy kids film may lucas trying perhaps perspective overly critical admittedly took original trilogy story somewhere modern mythology serious science fiction films something else something modern deep somehow thankfully endure wa

In [41]:
#E) Clearly print Performance Metrics.
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores(y_val, predicted):
    print('Accuracy score: ', accuracy_score(y_val, predicted))
    print('F1 score: ', f1_score(y_val, predicted, average='micro'))
    print('Average precision score: ', average_precision_score(y_val, predicted, average='micro'))
    print('Average recall score: ', recall_score(y_val, predicted, average='micro'))

In [42]:
print('Bag-of-words')
print_evaluation_scores(y_test, predicted_topic)

Bag-of-words
Accuracy score:  0.5514184397163121
F1 score:  0.8177726485862666
Average precision score:  0.7151916671295193
Average recall score:  0.7961797752808989


In [43]:
#4) Improve Performance of model. 
#A) Experiment with other vectorisers
#preprocessing with two methods
#Method1 =Normalization

def lower_case(text):
    return text.lower()
def remove_punctuation(text):
    return re.sub('[^a-zA-Z]',' ', str(text))

def normalize_document(text):
    text = remove_punctuation(text)
    text = lower_case(text)
    return text

In [44]:
df['normalize_document'] = df['text'].apply(normalize_document)
df.head() 

C:\Users\HP\AppData\Local\Temp/ipykernel_97244/567325569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['normalize_document'] = df['text'].apply(normalize_document)


,text,topic,normalize_document
0,info found pages mb pdf files wait untill team...,Student,info found pages mb pdf files wait untill team...
2,het kader van kernfusie op aarde maak je eigen...,Student,het kader van kernfusie op aarde maak je eigen...
3,testing testing,Student,testing testing
4,thanks yahoo toolbar capture urls popups means...,InvestmentBanking,thanks yahoo toolbar capture urls popups means...
5,interesting conversation dad morning talking k...,InvestmentBanking,interesting conversation dad morning talking k...


In [45]:
df['Method1'] = df['normalize_document'].apply(lambda x : nltk.word_tokenize(x))
df.head()

C:\Users\HP\AppData\Local\Temp/ipykernel_97244/275242146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Method1'] = df['normalize_document'].apply(lambda x : nltk.word_tokenize(x))


,text,topic,normalize_document,Method1
0,info found pages mb pdf files wait untill team...,Student,info found pages mb pdf files wait untill team...,"[info, found, pages, mb, pdf, files, wait, unt..."
2,het kader van kernfusie op aarde maak je eigen...,Student,het kader van kernfusie op aarde maak je eigen...,"[het, kader, van, kernfusie, op, aarde, maak, ..."
3,testing testing,Student,testing testing,"[testing, testing]"
4,thanks yahoo toolbar capture urls popups means...,InvestmentBanking,thanks yahoo toolbar capture urls popups means...,"[thanks, yahoo, toolbar, capture, urls, popups..."
5,interesting conversation dad morning talking k...,InvestmentBanking,interesting conversation dad morning talking k...,"[interesting, conversation, dad, morning, talk..."


In [46]:
df.drop(columns=['normalize_document'], axis=1, inplace=True)
df.head()

F:\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text,topic,Method1
0,info found pages mb pdf files wait untill team...,Student,"[info, found, pages, mb, pdf, files, wait, unt..."
2,het kader van kernfusie op aarde maak je eigen...,Student,"[het, kader, van, kernfusie, op, aarde, maak, ..."
3,testing testing,Student,"[testing, testing]"
4,thanks yahoo toolbar capture urls popups means...,InvestmentBanking,"[thanks, yahoo, toolbar, capture, urls, popups..."
5,interesting conversation dad morning talking k...,InvestmentBanking,"[interesting, conversation, dad, morning, talk..."


In [47]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords

In [48]:
import gensim
from nltk import word_tokenize
from nltk import sent_tokenize
import re
from bs4 import BeautifulSoup

In [49]:
from keras.preprocessing.text import Tokenizer

text =df['Method1']
sentence = []
for i in text:
    sentence.append(i)

# using tokenizer 
model = Tokenizer()
model.fit_on_texts(sentence)

#print keys 
keys = list(model.word_index.keys())
print(f'Key : {keys[0:20]}')
print('Total_Keys:', len(keys)) 
print('-'*40)

#create bag of words representation 
bow = model.texts_to_matrix(sentence, mode='count')
print(bow)

Key : ['like', 'one', 'get', 'know', 'think', 'go', 'would', 'time', 'really', 'well', 'good', 'people', 'urllink', 'got', 'nbsp', 'back', 'much', 'see', 'going', 'day']
Total_Keys: 25395
----------------------------------------
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0. 15. 14. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  1.  0. ...  0.  0.  0.]
 [ 0.  0.  1. ...  1.  0.  0.]
 [ 0.  1.  0. ...  0.  1.  1.]]


In [50]:
#TF-IDF Vectorizer
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(analyzer='word',
                        tokenizer=dummy_fun,
                        preprocessor=dummy_fun,
                        token_pattern=None,
                        stop_words = 'english') 

In [51]:
tfidf_method1 = tfidf.fit_transform(df.Method1)
tfidf_method1.shape

F:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


(2817, 25208)

In [52]:
tfidf_df_method1 = pd.DataFrame(tfidf_method1.toarray(),
             columns = tfidf.get_feature_names(),
             index = df.index)

F:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [53]:
tfidf_df_method1.mean().sort_values(ascending = False).head(10) 

urllink    0.024477
like       0.023928
know       0.017468
time       0.015379
think      0.015340
good       0.015107
really     0.014560
nbsp       0.014527
people     0.014080
got        0.013148
dtype: float64

In [54]:
#C) Tune Parameters/Hyperparameters of the model/s. [
def sentence(text):
    sent = ' '.join(text)
    #for i in txt:
     #   sent.append(' '.join(i))
    return sent

In [55]:
df['Method1'] = df['Method1'].apply(sentence)
df.head()

C:\Users\HP\AppData\Local\Temp/ipykernel_97244/3459411117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Method1'] = df['Method1'].apply(sentence)


,text,topic,Method1
0,info found pages mb pdf files wait untill team...,Student,info found pages mb pdf files wait untill team...
2,het kader van kernfusie op aarde maak je eigen...,Student,het kader van kernfusie op aarde maak je eigen...
3,testing testing,Student,testing testing
4,thanks yahoo toolbar capture urls popups means...,InvestmentBanking,thanks yahoo toolbar capture urls popups means...
5,interesting conversation dad morning talking k...,InvestmentBanking,interesting conversation dad morning talking k...


In [56]:
X_train, X_test, y_train, y_test = train_test_split(df['Method1'].values, df['topic'].values, test_size=0.15, random_state=42)
print('Training utterances: {}'.format(X_train.shape[0]))
print('Validation utterances: {}'.format(X_test.shape[0]))

Training utterances: 2394
Validation utterances: 423


In [57]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer()

In [58]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)
X_train, X_test

(<2394x23468 sparse matrix of type '<class 'numpy.float64'>'
 	with 154453 stored elements in Compressed Sparse Row format>,
 <423x23468 sparse matrix of type '<class 'numpy.float64'>'
 	with 23193 stored elements in Compressed Sparse Row format>)

In [59]:
#B) Build classifier Models using other algorithms than base model
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=5000)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

X_train, X_test

(<2394x5000 sparse matrix of type '<class 'numpy.float64'>'
 	with 83369 stored elements in Compressed Sparse Row format>,
 <423x5000 sparse matrix of type '<class 'numpy.float64'>'
 	with 12836 stored elements in Compressed Sparse Row format>)

In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model_mnb_m1 = MultinomialNB()
model_mnb_m1.fit(X_train, y_train)
pred = model_mnb_m1.predict(X_test)
print(accuracy_score(y_test, pred))

0.5224586288416075


In [61]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
print('RF_Score:', accuracy_score(y_test, pred))

RF_Score: 0.6264775413711584


In [62]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
pred = bnb.predict(X_test)
print('NB_Score:', accuracy_score(y_test, pred))

NB_Score: 0.6335697399527187


In [63]:
#D) Clearly print Performance Metrics. 
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

                      precision    recall  f1-score   support

          Accounting       0.00      0.00      0.00         1
             Banking       0.00      0.00      0.00         2
    BusinessServices       0.00      0.00      0.00         2
Communications-Media       0.00      0.00      0.00         2
           Education       0.50      0.20      0.29        25
         Engineering       1.00      0.06      0.11        17
            Internet       0.00      0.00      0.00         2
   InvestmentBanking       0.50      0.10      0.17        10
          Non-Profit       0.00      0.00      0.00         9
             Science       0.00      0.00      0.00         6
   Sports-Recreation       0.80      0.50      0.62         8
             Student       0.58      0.43      0.50        58
          Technology       0.70      0.94      0.80       214
              indUnk       0.42      0.46      0.44        67

            accuracy                           0.63       423
      

F:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
#5) Share insights on relative performance comparison
#A) Which vectorizer performed better? Probable reason?. 
#TF-IDF Vectorizer performed better in Accuracy, but when compared to precision, f1-score and recall Count Vectorizer performed better than TF-IDF Vectorizer.

In [65]:
#B) Which model outperformed? Probable reason?
# Out of all the models performed from Count vectorizer, Bernoulli Naive bayes model from TF-IDF vectorizer performed better with 63% accuracy.

In [66]:
#C. Which parameter/hyperparameter significantly helped to improve performance?Probable reason?
#MultiLabelBinarizer helped to improve performance more accurately, because it predicted accurately same as true topic.
#BernoulliNB helped to improve performance compared to RandomForest,MultinomialNB

In [68]:
#D) According to you, which performance metric should be given most importance, why?. 
# Count Vectorier can be given most importance, because it evaluation metrics has better performance compared to TF-IDF vectorizer